# Large File Input

Source:  [https://github.com/d-insight/code-bank.git](https://github.com/d-insight/code-bank.git)  
License: [MIT License](https://opensource.org/licenses/MIT). See open source [license](LICENSE) in the Code Bank repository. 

---

### Import libraries

In [1]:
import numpy as np
import pandas as pd
from pathlib import Path
import csv
from pprint import pprint 
import sqlite3
from sqlalchemy import create_engine

### Set data directory

In [2]:
DIR = 'data'
FILE = '/yellow_tripdata_2016-03.csv'

csv_file = '{}{}'.format(DIR, FILE)

print('File directory: {}'.format(csv_file))

File directory: data/yellow_tripdata_2016-03.csv


#### How large is the dataset on disk?

In [3]:
size = Path(csv_file).stat().st_size

In [4]:
f'{size:,} bytes'

'1,914,669,757 bytes'

#### How many rows of data?

In [5]:
with open(csv_file) as f:
    row_count = sum(1 for row in f)     # generator expression

In [6]:
f'{row_count:,} rows'

'12,210,953 rows'

#### Inspect Raw Data

In [7]:
with open(csv_file) as f:
    for i, row in enumerate(f):
        pprint(row)
        if i == 3:
            break

'VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount\n'
('1,2016-03-01 00:00:00,2016-03-01 '
 '00:07:55,1,2.50,-73.97674560546875,40.765151977539062,1,N,-74.004264831542969,40.746128082275391,1,9,0.5,0.5,2.05,0,0.3,12.35\n')
('1,2016-03-01 00:00:00,2016-03-01 '
 '00:11:06,1,2.90,-73.983482360839844,40.767925262451172,1,N,-74.005943298339844,40.733165740966797,1,11,0.5,0.5,3.05,0,0.3,15.35\n')
('2,2016-03-01 00:00:00,2016-03-01 '
 '00:31:06,2,19.98,-73.782020568847656,40.644809722900391,1,N,-73.974540710449219,40.675769805908203,1,54.5,0.5,0.5,8,0,0.3,63.8\n')


Cleaner view

In [8]:
with open(csv_file) as f:
    csv_dict = csv.DictReader(f)
    for i, row in enumerate(csv_dict):
        pprint(row)
        if i ==3:
            break

{'RatecodeID': '1',
 'VendorID': '1',
 'dropoff_latitude': '40.746128082275391',
 'dropoff_longitude': '-74.004264831542969',
 'extra': '0.5',
 'fare_amount': '9',
 'improvement_surcharge': '0.3',
 'mta_tax': '0.5',
 'passenger_count': '1',
 'payment_type': '1',
 'pickup_latitude': '40.765151977539062',
 'pickup_longitude': '-73.97674560546875',
 'store_and_fwd_flag': 'N',
 'tip_amount': '2.05',
 'tolls_amount': '0',
 'total_amount': '12.35',
 'tpep_dropoff_datetime': '2016-03-01 00:07:55',
 'tpep_pickup_datetime': '2016-03-01 00:00:00',
 'trip_distance': '2.50'}
{'RatecodeID': '1',
 'VendorID': '1',
 'dropoff_latitude': '40.733165740966797',
 'dropoff_longitude': '-74.005943298339844',
 'extra': '0.5',
 'fare_amount': '11',
 'improvement_surcharge': '0.3',
 'mta_tax': '0.5',
 'passenger_count': '1',
 'payment_type': '1',
 'pickup_latitude': '40.767925262451172',
 'pickup_longitude': '-73.983482360839844',
 'store_and_fwd_flag': 'N',
 'tip_amount': '3.05',
 'tolls_amount': '0',
 'total

#### Import few rows using pandas

In [9]:
print(pd.read_csv(csv_file, nrows=2))

   VendorID tpep_pickup_datetime tpep_dropoff_datetime  passenger_count  \
0         1  2016-03-01 00:00:00   2016-03-01 00:07:55                1   
1         1  2016-03-01 00:00:00   2016-03-01 00:11:06                1   

   trip_distance  pickup_longitude  pickup_latitude  RatecodeID  \
0            2.5        -73.976746        40.765152           1   
1            2.9        -73.983482        40.767925           1   

  store_and_fwd_flag  dropoff_longitude  dropoff_latitude  payment_type  \
0                  N         -74.004265         40.746128             1   
1                  N         -74.005943         40.733166             1   

   fare_amount  extra  mta_tax  tip_amount  tolls_amount  \
0            9    0.5      0.5        2.05             0   
1           11    0.5      0.5        3.05             0   

   improvement_surcharge  total_amount  
0                    0.3         12.35  
1                    0.3         15.35  


Loaded succesfully!

### Calculating Size

Let’s load the first 10,000 lines into pandas and measure the amount of memory being used. Then we can calculate the total amount of memory needed to load the complete file

In [10]:
nrows = 10_000
MB = 2**20  # 1 MB = 2**20 bytes = 1,048,576 bytes
df = pd.read_csv(csv_file, nrows=nrows)
df_mb = df.memory_usage(deep=True).sum() / MB 
df_mb          # how much memory the DataFrame is consuming in MB

3.299835205078125

In [11]:
df_mb * (row_count / nrows)     # total memory consumption for the whole data file. ~4GB

4029.4132596954346

### Import

#### Standard way -> Memory Error!

In [12]:
#df = pd.read_csv(file)

In [13]:
df.shape

(10000, 19)

<img src="img/memoryError.png">

In [1]:
#df.info()

#### Solution A: Chunking!

In [15]:
chunk_size=50000
batch_no=1
for chunk in pd.read_csv(csv_file,chunksize=chunk_size):
    chunk.to_csv(DIR+'chunk'+str(batch_no)+'.csv',index=False)
    batch_no+=1

Importing a single chunk file into pandas dataframe:

In [17]:
DIR

'data'

In [18]:
df1 = pd.read_csv(DIR+'chunk2.csv')
df1.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,2,2016-03-10 11:52:03,2016-03-10 12:14:00,1,2.26,-73.975624,40.758133,1,N,-73.950508,40.773960,2,14.5,0.0,0.5,0.00,0.0,0.3,15.30
1,2,2016-03-10 11:52:03,2016-03-10 12:05:57,1,2.24,-73.956642,40.784100,1,N,-73.983032,40.775478,2,11.0,0.0,0.5,0.00,0.0,0.3,11.80
2,2,2016-03-10 11:52:03,2016-03-10 11:56:32,1,0.48,-73.988281,40.748219,1,N,-73.985207,40.744888,1,4.5,0.0,0.5,1.59,0.0,0.3,6.89
3,2,2016-03-10 11:52:03,2016-03-10 12:15:03,1,2.30,-73.990524,40.756641,1,N,-73.991379,40.750637,2,15.5,0.0,0.5,0.00,0.0,0.3,16.30
4,2,2016-03-10 11:52:04,2016-03-10 12:01:55,1,1.06,-73.982101,40.778446,1,N,-73.975639,40.782120,2,8.0,0.0,0.5,0.00,0.0,0.3,8.80


#### Solution B: SQL!

1. Create a Connector to a Database
2. Build the Database (load CSV File) by Chunking
3. Construct Pandas DF from Database using SQL query

In [19]:
# Create a Connector to a Database (creates it in Memory - not an SQL db)
csv_database = create_engine('sqlite:///csv_database.db')

In [20]:
# Build the Database (load CSV File) by Chunking
chunk_size=10000
i=0
j=0

for df in pd.read_csv(csv_file, chunksize=chunk_size, iterator=True):
    df = df.rename(columns= {c: c.replace(' ','') for c in df.columns})
    # remove spaces in col names is important to make SQL run well
    df.index += j
    
# Put all data in data_sql Table (temp table), using engine to fire it in background. Temp db sitting in Memory
    df.to_sql('data_sql', csv_database, if_exists = 'append')    
    j = df.index[-1]+1
    
    print('| index: {}'.format(j))

dex: 223660000
| index: 225780000
| index: 227910000
| index: 230050000
| index: 232200000
| index: 234360000
| index: 236530000
| index: 238710000
| index: 240900000
| index: 243100000
| index: 245310000
| index: 247530000
| index: 249760000
| index: 252000000
| index: 254250000
| index: 256510000
| index: 258780000
| index: 261060000
| index: 263350000
| index: 265650000
| index: 267960000
| index: 270280000
| index: 272610000
| index: 274950000
| index: 277300000
| index: 279660000
| index: 282030000
| index: 284410000
| index: 286800000
| index: 289200000
| index: 291610000
| index: 294030000
| index: 296460000
| index: 298900000
| index: 301350000
| index: 303810000
| index: 306280000
| index: 308760000
| index: 311250000
| index: 313750000
| index: 316260000
| index: 318780000
| index: 321310000
| index: 323850000
| index: 326400000
| index: 328960000
| index: 331530000
| index: 334110000
| index: 336700000
| index: 339300000
| index: 341910000
| index: 344530000
| index: 3471600

In [21]:
df2 = pd.read_sql_query('SELECT * FROM data_sql WHERE VendorID=1 AND trip_distance=2.90', csv_database)

In [22]:
df2

,index,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,RatecodeID,store_and_fwd_flag,dropoff_longitude,dropoff_latitude,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,1,1,2016-03-01 00:00:00,2016-03-01 00:11:06,1,2.9,-73.983482,40.767925,1,N,-74.005943,40.733166,1,11.0,0.5,0.5,3.05,0.0,0.3,15.35
1,142,1,2016-03-01 00:00:52,2016-03-01 00:12:24,1,2.9,-73.985558,40.741905,1,N,-73.955894,40.774605,1,11.5,0.5,0.5,2.55,0.0,0.3,15.35
2,275085,1,2016-03-01 00:03:30,2016-03-01 00:16:36,1,2.9,-73.983765,40.768234,1,N,-74.000679,40.731564,1,12.0,0.5,0.5,2.65,0.0,0.3,15.95
3,275092,1,2016-03-01 00:03:33,2016-03-01 00:12:11,2,2.9,-73.987823,40.722610,1,N,-73.941551,40.711826,2,10.5,0.5,0.5,0.00,0.0,0.3,11.80
4,275257,1,2016-03-01 00:04:44,2016-03-01 00:19:19,1,2.9,-74.007751,40.740673,1,N,-73.987473,40.765743,1,13.0,0.5,0.5,2.14,0.0,0.3,16.44
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64690,7472520324,1,2016-03-31 23:57:57,2016-04-01 00:13:28,1,2.9,-73.968697,40.759853,1,N,-73.930763,40.757896,1,14.0,0.5,0.5,2.30,0.0,0.3,17.60
64691,7472520507,1,2016-03-31 23:58:32,2016-04-01 00:12:36,1,2.9,-73.958412,40.760708,1,N,-73.991592,40.744789,1,12.0,0.5,0.5,1.00,0.0,0.3,14.30
64692,7472520540,1,2016-03-31 23:58:37,2016-04-01 00:13:38,2,2.9,-73.982666,40.750977,1,N,-74.005173,40.715492,2,12.5,0.5,0.5,0.00,0.0,0.3,13.80
64693,7472520623,1,2016-03-31 23:58:56,2016-04-01 00:04:09,1,2.9,-73.793320,40.644867,1,N,-73.782173,40.666496,2,10.0,0.5,0.5,0.00,0.0,0.3,11.30


In [23]:
df2.columns

Index(['index', 'VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'pickup_longitude',
       'pickup_latitude', 'RatecodeID', 'store_and_fwd_flag',
       'dropoff_longitude', 'dropoff_latitude', 'payment_type', 'fare_amount',
       'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')